In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

In [ ]:
# import pandas as pd

# # Load the data
# data = pd.read_csv('/kaggle/working/GT.csv')

# # Remove all rows after the 3000th row
# data = data.iloc[:2000]

# # Save the updated data to a new CSV file
# data.to_csv('/kaggle/working/GT.csv', index=False)

In [ ]:
data = pd.read_csv('/kaggle/input/isicgt25k/ISIC_2019_Training_GroundTruth.csv')
data.head()

In [ ]:
store_list = []
image_height = 350
image_width = 350
for i in tqdm(range(data.shape[0])):
    path = '/kaggle/input/isic2019/ISIC_2019_Training_Input/' + data['image'][i] + '.jpg'
    image_check = image.load_img(path, target_size=(image_height, image_width))
    image_check = image.img_to_array(image_check)
    # scaling the images
    image_check = image_check/255
    store_list.append(image_check)

In [ ]:
x = np.array(store_list)
y = data.drop(columns=['image'])
Y = y.to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, Y, random_state=42, test_size=0.20)


In [ ]:
# Define the autoencoder architecture
input_img = Input(shape=x_train[0].shape)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPool2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPool2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPool2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(x_train, x_train, epochs=10, batch_size=128, validation_data=(x_test, x_test))

# Use the encoder part of the autoencoder to extract features
encoder = Model(input_img, encoded)
features_train = encoder.predict(x_train)
features_test = encoder.predict(x_test)

# Define the classification model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=features_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(9, activation='sigmoid'))
model.summary()

In [ ]:
# Train the classification model
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(features_train, y_train, epochs=10, batch_size=128, validation_data=(features_test, y_test))

In [ ]:
score = model.evaluate(features_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Make predictions on the test set
y_pred = model.predict(features_test)

# Print the actual and predicted data side by side
for i in range(len(y_test)):
    print('Actual:', y_test[i], 'Predicted:', y_pred[i])

# Plot the accuracy and loss graphs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()